In [95]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk import edit_distance as ed
from scipy.spatial.distance import cosine
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import FastText
import torch
#import torchtext
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pandas as pd

In [106]:
articles = pd.read_csv("shared_articles.csv", delimiter= ',')
interactions = pd.read_csv("users_interactions.csv", delimiter= ',')

articles
interactions.head(30)



,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


In [118]:
#1
#Pre-processing, labeling event ratings

interactions['eventRating'] = ''

for session in interactions['sessionId']:
    df1= interactions[interactions['sessionId']==session]
    for content in df1['contentId']:
        df = df1[df1['contentId']==content]
        if df.empty:
            continue
        idx = df.index.values.tolist()
        if (df['eventType'].str.contains("LIKE")).any():
            interactions.loc[idx, 'eventRating'] = 5.0
        elif ((df['eventType'].str.contains("FOLLOW")).any() & (df['eventType'].str.contains("BOOKMARK")).any()):
            interactions.loc[idx, 'eventRating'] = 4.0
        elif ((df['eventType'].str.contains("FOLLOW")).any() | (df['eventType'].str.contains("BOOKMARK")).any() & ~((df['eventType'].str.contains("FOLLOW")).any() & (df['eventType'].str.contains("BOOKMARK")).any())):
            interactions.loc[idx, 'eventRating'] = 3.0
        elif ((df['eventType'].str.contains("COMMENT CREATED")).any() & ~((df['eventType'].str.contains("BOOKMARK")).any() | (df['eventType'].str.contains("FOLLOW")).any() | (df['eventType'].str.contains("LIKE")).any())):
            interactions.loc[idx, 'eventRating'] = 2.0
        elif ((df['eventType'].str.contains("VIEW")).any() & ~((df['eventType'].str.contains("COMMENT CREATED")).any() | (df['eventType'].str.contains("BOOKMARK")).any() | (df['eventType'].str.contains("FOLLOW")).any() | (df['eventType'].str.contains("LIKE")).any())):
            interactions.loc[idx,'eventRating'] = 1.0           

interactions["eventRating"].head(10)

0    1.0
1    1.0
2    5.0
3    5.0
4    1.0
5    5.0
6    1.0
7    1.0
8    5.0
9    1.0
Name: eventRating, dtype: object

In [145]:
#1
#Selecting articles and users with enough interactions

users_interactions_count = interactions.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count))
users_with_enough_interactions_df = users_interactions_count[users_interactions_count >= 3].reset_index()[['personId']]

article_interactions_count = interactions.groupby(['personId', 'contentId']).size().groupby('contentId').size()
articles_with_enough_interactions_df = article_interactions_count[article_interactions_count >= 2].reset_index()[['contentId']]

print("articles with at least 2 interactions:", len(articles_with_enough_interactions_df))
print("users with at least 3 interactions:", len(users_with_enough_interactions_df))
pre_processed_df = interactions[(interactions.index.isin(users_with_enough_interactions_df.index)) & interactions.index.isin(articles_with_enough_interactions_df.index)]

#Spliting the data to training and testing sets
train_set, test_set = train_test_split(pre_processed_df, train_size=0.80)
print(test_set["eventRating"])

# users: 1895
articles with at least 2 interactions: 2744
users with at least 3 interactions: 1400
1341    5.0
736     1.0
489     3.0
822     3.0
679     1.0
       ... 
825     1.0
1135    1.0
6       1.0
1398    5.0
777     1.0
Name: eventRating, Length: 280, dtype: object
